In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [88]:
import joblib
import lightgbm as lgb
from math import radians, cos, sin, asin, sqrt
import numpy as np
import pandas as pd

import random
from random import choice

<IPython.core.display.Javascript object>

In [39]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km

<IPython.core.display.Javascript object>

In [68]:
# data for dataset generation

categories = {0: "food", 1: "documents", 2: "parcel", 3: "flowers", 4: "other"}

rating_choice = np.linspace(3, 5)

categories_choice = [1, 0]

current_hub = np.arange(1, 4)

COURIERS_NUM = 500

# got from another module
hubs = {0: [55.371665, 39.073222], 1: [55.470839, 39.080874], 2: [55.382143, 39.030079]}

<IPython.core.display.Javascript object>

In [69]:
couriers = pd.DataFrame(
    {
        # "phone_number": [78373658431 for i in range(COURIERS_NUM)],
        "rating": [choice(rating_choice) for i in range(COURIERS_NUM)],
        "current_hub": [choice(current_hub) for i in range(COURIERS_NUM)],
        "food_category": [choice(categories_choice) for i in range(COURIERS_NUM)],
        "documents_category": [choice(categories_choice) for i in range(COURIERS_NUM)],
        "parcel_category": [choice(categories_choice) for i in range(COURIERS_NUM)],
        "flowers_category": [choice(categories_choice) for i in range(COURIERS_NUM)],
        "other_category": [choice(categories_choice) for i in range(COURIERS_NUM)],
        "mean_speed": np.linspace(5.0, 7.0, COURIERS_NUM),
        "orders_in_line": 0,
        "rapid": [0 if i % 2 == 0 else 1 for i in range(COURIERS_NUM)],
        "type": [choice(current_hub) for i in range(COURIERS_NUM)],
    }
)

<IPython.core.display.Javascript object>

In [70]:
couriers

,rating,current_hub,food_category,documents_category,parcel_category,flowers_category,other_category,mean_speed,orders_in_line,rapid,type
0,3.326531,1,0,0,1,1,1,5.000000,0,0,3
1,3.489796,1,0,1,1,0,1,5.004008,0,1,2
2,4.020408,1,0,0,1,0,0,5.008016,0,0,2
3,3.571429,2,1,0,0,0,0,5.012024,0,1,1
4,3.448980,1,0,0,1,0,1,5.016032,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
495,4.632653,2,0,1,0,1,0,6.983968,0,1,1
496,4.142857,3,1,1,0,0,1,6.987976,0,0,1
497,4.142857,2,1,0,0,1,0,6.991984,0,1,1
498,3.448980,1,0,0,1,1,1,6.995992,0,0,1


<IPython.core.display.Javascript object>

In [71]:
couriers.current_hub = couriers.apply(
    lambda x: np.nan if x.rapid == 1 else x.current_hub, axis=1
)

<IPython.core.display.Javascript object>

In [72]:
# get info about orders
orders = pd.read_excel("data/dataset_orders_final_new.xlsx")

<IPython.core.display.Javascript object>

In [73]:
orders

,date,adress from,latitude from,longtitude from,adress to,latitude to,longtitude to,rapid,order_type,cost,cashed
0,2020-07-19 01:03:57.548,Московская область Егорьевск Улица Благонравова 1,55.380317,39.021655,Московская область Егорьевск Улица 14 лет Октя...,55.363878,39.043309,0,1,3741,1
1,2020-07-19 01:03:57.548,Московская область Егорьевск Бульвар Ларюшина 1,55.379765,39.051152,Московская область Егорьевск Комсомольская ули...,55.465594,39.053360,1,2,1402,0
2,2020-07-19 01:03:57.548,Московская область Егорьевск 3-й Железнодорожн...,55.380242,39.012773,Московская область Егорьевск Улица Алексея Туп...,55.385411,39.032033,0,1,694,1
3,2020-07-19 01:03:57.548,Московская область Егорьевск Улица Самойловой 1,55.378710,39.021465,Московская область Егорьевск Улица Комарова 1,55.383617,39.059115,1,2,2485,1
4,2020-07-19 01:03:57.548,Московская область Егорьевск Улица Козлова 1,55.369176,39.023661,Московская область Егорьевск 3-я Нечаевская ул...,55.400864,39.039519,0,3,3726,0
...,...,...,...,...,...,...,...,...,...,...,...
4543,2020-09-17 01:03:57.548,Московская область Егорьевск Ширяевский проезд 1,55.397313,39.042935,Московская область Егорьевск Улица Софьи Перов...,55.388530,39.037743,1,2,2556,0
4544,2020-09-17 01:03:57.548,Московская область Егорьевск Улица Самойловой 1,55.378710,39.021465,Московская область Егорьевск Поселковая улица 1,55.369250,39.040974,0,2,2021,1
4545,2020-09-17 01:03:57.548,Московская область Егорьевск Проезд Радугина 1,55.377347,39.085151,Московская область Егорьевск Колхозная улица 1,55.380851,39.014738,0,3,4594,0
4546,2020-09-17 01:03:57.548,Московская область Егорьевск Улица Майора Удач...,55.365701,39.051464,Московская область Егорьевск Дачная улица 1,55.372221,39.032547,0,2,479,0


<IPython.core.display.Javascript object>

In [78]:
def primary_selection(order, couriers):
    """
    strict selection of couriers according to the parameters:
    urgency, nearest hub, correspondence of the category of
    goods to the capabilities of the courier and checking for
    ratings for unpaid goods and goods cost more than 2000
    """
    if order.rapid == 1:
        selected_couriers = couriers[couriers.rapid == 1]
    else:
        selected_couriers = couriers[couriers.rapid == 0]
        distances = [
            haversine(
                order["latitude from"], order["longtitude from"], hubs[i][0], hubs[i][1]
            )
            for i in range(3)
        ]
        selected_couriers = selected_couriers[
            selected_couriers.current_hub == np.argmin(distances)
        ]

    selected_couriers = selected_couriers[
        selected_couriers[categories[order.order_type] + "_category"] == 1
    ]

    if order.cost > 2000 or order.cashed == 0:
        selected_couriers = selected_couriers[selected_couriers.rating > 4.0]

    return selected_couriers

<IPython.core.display.Javascript object>

In [79]:
selected = primary_selection(orders.loc[6], couriers)

<IPython.core.display.Javascript object>

In [80]:
selected

,rating,current_hub,food_category,documents_category,parcel_category,flowers_category,other_category,mean_speed,orders_in_line,rapid,type
7,4.632653,NaN,1,1,0,0,0,5.028056,0,1,1
31,4.591837,NaN,1,1,0,0,1,5.124248,0,1,1
39,4.265306,NaN,1,1,0,0,0,5.156313,0,1,2
57,4.183673,NaN,1,0,0,1,1,5.228457,0,1,2
69,4.142857,NaN,1,1,1,1,1,5.276553,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...
453,4.551020,NaN,1,1,0,0,0,6.815631,0,1,1
455,4.510204,NaN,1,0,0,1,0,6.823647,0,1,1
465,4.387755,NaN,1,0,1,0,1,6.863727,0,1,2
479,4.714286,NaN,1,0,1,0,0,6.919840,0,1,1


<IPython.core.display.Javascript object>

In [81]:
import lightgbm as lgb

gbm = lgb.LGBMRanker()

<IPython.core.display.Javascript object>

In [82]:
y = couriers.rating.copy().apply(round)
X = couriers.copy()
X = X.drop(columns=["rating"])

<IPython.core.display.Javascript object>

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=1
)

<IPython.core.display.Javascript object>

In [84]:
query_train = [X_train.shape[0]]
query_val = [X_val.shape[0]]
query_test = [X_test.shape[0]]

<IPython.core.display.Javascript object>

In [85]:
gbm.fit(
    X_train,
    y_train,
    group=query_train,
    eval_set=[(X_val, y_val)],
    eval_group=[query_val],
    eval_at=[5, 10, 20],
    early_stopping_rounds=50,
)

[1]	valid_0's ndcg@5: 0.522241	valid_0's ndcg@10: 0.524034	valid_0's ndcg@20: 0.528029
Training until validation scores don't improve for 50 rounds
[2]	valid_0's ndcg@5: 0.731671	valid_0's ndcg@10: 0.75884	valid_0's ndcg@20: 0.652761
[3]	valid_0's ndcg@5: 0.758943	valid_0's ndcg@10: 0.702838	valid_0's ndcg@20: 0.63669
[4]	valid_0's ndcg@5: 0.819196	valid_0's ndcg@10: 0.718377	valid_0's ndcg@20: 0.663862
[5]	valid_0's ndcg@5: 0.755627	valid_0's ndcg@10: 0.730924	valid_0's ndcg@20: 0.65217
[6]	valid_0's ndcg@5: 0.767134	valid_0's ndcg@10: 0.738391	valid_0's ndcg@20: 0.656547
[7]	valid_0's ndcg@5: 0.886915	valid_0's ndcg@10: 0.761023	valid_0's ndcg@20: 0.670458
[8]	valid_0's ndcg@5: 0.834333	valid_0's ndcg@10: 0.727036	valid_0's ndcg@20: 0.631834
[9]	valid_0's ndcg@5: 0.766614	valid_0's ndcg@10: 0.722735	valid_0's ndcg@20: 0.629235
[10]	valid_0's ndcg@5: 0.868712	valid_0's ndcg@10: 0.750509	valid_0's ndcg@20: 0.646983
[11]	valid_0's ndcg@5: 0.886915	valid_0's ndcg@10: 0.758024	valid_0's n

LGBMRanker()

<IPython.core.display.Javascript object>

In [86]:
test_pred = gbm.predict(X_test)

<IPython.core.display.Javascript object>

In [87]:
X_test["predicted_ranking"] = test_pred
X_test.sort_values("predicted_ranking", ascending=False).iloc[0]

/home/maria/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


current_hub                NaN
food_category         1.000000
documents_category    0.000000
parcel_category       1.000000
flowers_category      0.000000
other_category        1.000000
mean_speed            6.446894
orders_in_line        0.000000
rapid                 1.000000
type                  1.000000
predicted_ranking     1.171083
Name: 361, dtype: float64

<IPython.core.display.Javascript object>

In [89]:
joblib.dump(gbm, "lgb.pkl")

['lgb.pkl']

<IPython.core.display.Javascript object>